## Import Bibliotek

In [476]:
import numpy as np

## Wczytanie danych

In [477]:
nazwa_pliku = 'input.txt'

# Funkcja, która zastępuje przecinki jako separator dziesiętny na kropki - w celu poprawnego wczytania danych za
# pomocą np.loadtxt
def replace_commas_with_periods(filename):
    with open(filename, 'r') as file:
        contents = file.read()
        new_contents = contents.replace(',', '.')
    with open(filename, 'w') as file:
        file.write(new_contents)

replace_commas_with_periods(nazwa_pliku)
coords = np.loadtxt('./' + nazwa_pliku, delimiter=' ')
print("Dane wejściowe:")
print(coords)

Dane wejściowe:
[[-2.5  16.5 ]
 [-2.   19.  ]
 [-1.5  17.65]
 [-1.   13.8 ]
 [-0.5   8.8 ]
 [ 0.    4.  ]
 [ 0.5   0.75]]


## Przypisanie danych do zmiennych

In [478]:
xi = coords[:, 0]
fx = coords[:, 1]

## Funkcja która oblicza wartość następnej kolumny na podstawie danych z poprzedniej

In [479]:
def get_next_column(fx: np.ndarray):
   next_column = np.array([])
   last_f = fx[-1]
   for index, f in np.ndenumerate(fx):
    if f != last_f:
        next_column = np.append(next_column, round((fx[index[0]+1] - f), 2))
   return next_column
print("Sprawdzenie poprawności działania funkcji:")
print(get_next_column(fx))

Sprawdzenie poprawności działania funkcji:
[ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]


## Funkcja która oblicza kolejne kolumny w tabeli

In [480]:
columns = []

def calculate_cols(column: np.ndarray):
    global columns
    col = get_next_column(column)
    columns.append(col)
    if len(np.unique(col)) != 1:
        calculate_cols(col)
    else:
        return

calculate_cols(fx)
print(columns)

[array([ 2.5 , -1.35, -3.85, -5.  , -4.8 , -3.25]), array([-3.85, -2.5 , -1.15,  0.2 ,  1.55]), array([1.35, 1.35, 1.35, 1.35])]


## Wyświetlenie wyników

In [481]:
for i in range(len(columns)):
    print(f'Column {i+1}: {columns[i]}')

Column 1: [ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]
Column 2: [-3.85 -2.5  -1.15  0.2   1.55]
Column 3: [1.35 1.35 1.35 1.35]


In [482]:
for column in columns:
    print(column[3])

-5.0
0.2
1.35


In [483]:
point4_fs = []
for column in columns:
    try:
        print(column[0])
        point4_fs.append(column[0])
    except IndexError:
        print("Nie ma 4 elementu w kolumnie")

h = 0.5

skladniki = []

for index, delta_f in enumerate(point4_fs):
    skladnik = 1/(index+1) * delta_f
    skladnik = skladnik if index % 2 == 0 else -skladnik
    skladniki.append(skladnik)

print(skladniki)
numpy_skladniki = np.array(skladniki)
print(1/h * numpy_skladniki.sum() )

2.5
-3.85
1.35
[2.5, 1.925, 0.45]
9.75


# Metoda róznicy centralnej

In [484]:
def calculate_central_x(xi: np.ndarray):
    global h
    new_arr = []
    for x in xi:
        new_arr.append(x)
        new_arr.append(x + 1/2 * h)
    new_arr = np.delete(new_arr, -1)
    return new_arr

central_x = calculate_central_x(xi)

def create_column_with_empty_values_between(fx: np.ndarray):
    nan_array = np.full(len(fx),np.nan)
    calculated_array = np.array([])
    for element in zip(fx, nan_array):
        calculated_array = np.append(calculated_array, element)
    calculated_array = np.delete(calculated_array, -1)
    return calculated_array

def calculate_columns_central(columns: np.ndarray):
    columns_with_empty_values = []
    for index, col in enumerate(columns):
        c = create_column_with_empty_values_between(col)
        for i in range(index+1):
            c = np.append(c, np.nan)
            c = np.insert(c, 0, np.nan)
        columns_with_empty_values.append(c)
    return columns_with_empty_values

# print(create_column_with_empty_values_between(fx))
# print(calculate_columns_central(columns))

central_columns = np.vstack([create_column_with_empty_values_between(fx), calculate_columns_central(columns)])

#central_columns = np.flip(central_columns, axis=1)




# DO POPRAWY

In [501]:
wybrany_punkt = 6
print(central_x[wybrany_punkt])
new_sum = []
for index, col in enumerate(central_columns):
    if np.isnan(col[wybrany_punkt]):
        new_sum.append(((col[wybrany_punkt-1] + col[wybrany_punkt+1]) / 2) * 1/(index*))

new_sum = 1/h * np.array(new_sum).sum()
print(new_sum)

-1.0
-6.1499999999999995


# Metoda róznicy wstecznej

In [486]:
fx = np.flip(fx)
def get_next_column_reverse(fx: np.ndarray):
   next_column = np.array([])
   first_f = fx[-1]
   for index, f in np.ndenumerate(fx):
    if f != first_f:
        next_column = np.append(next_column, round((f - fx[index[0]+1] ), 2))
   return next_column
print("Sprawdzenie poprawności działania funkcji:")
print(get_next_column_reverse(fx))

Sprawdzenie poprawności działania funkcji:
[-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]


In [487]:
columns = []

def calculate_cols_reverse(column: np.ndarray):
    global columns
    col = get_next_column_reverse(column)
    print(col)
    columns.append(col)
    if len(np.unique(col)) != 1:
        calculate_cols_reverse(col)
    else:
        return

calculate_cols_reverse(fx)


[-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]
[ 1.55  0.2  -1.15 -2.5  -3.85]
[1.35 1.35 1.35 1.35]


In [488]:
for i in range(len(columns)):
    print(f'Column {i+1}: {columns[i]}')

Column 1: [-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]
Column 2: [ 1.55  0.2  -1.15 -2.5  -3.85]
Column 3: [1.35 1.35 1.35 1.35]


In [489]:
for column in columns:
    print(column[3])

-3.85
-2.5
1.35
